In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import statsmodels.api as sm
from statsmodels.regression.mixed_linear_model import MixedLM
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
# load in the data
genotypes = pd.read_csv('/content/drive/My Drive/comp_gen/Genotypic_data_maf10_min10_291acc.txt', index_col=0)
phenotype = pd.read_csv('/content/drive/My Drive/comp_gen/phenodata_BLUP_2012.txt', sep='\t', index_col='ID')

In [ ]:
def calculate_maf(df):
    # Calculate minor allele frequency
    maf = df.apply(lambda x: min(x.mean(), 1-x.mean()), axis=0)
    return maf

def ld_pruning(df, threshold=0.5):
    # Calculate correlation matrix
    corr = df.corr()
    # Identify pairs of SNPs with correlation greater than the threshold
    # Avoid double removal and self-comparison (i.e., diagonal elements)
    to_remove = set()
    for i in range(corr.shape[0]):
        for j in range(i+1, corr.shape[0]):
            if corr.iloc[i, j] > threshold:
                to_remove.add(corr.columns[j])
    return df.drop(columns=to_remove)

In [ ]:
# Apply MAF filtering
maf = calculate_maf(genotypes)
maf_threshold = 0.01  # Set MAF threshold
genotypes_filtered = genotypes.loc[:, maf >= maf_threshold]

# Apply LD pruning
genotypes_pruned = ld_pruning(genotypes_filtered, threshold=0.5)

In [ ]:
# 'genotypes' and 'phenotype' are already loaded and aligned by their indices
# now we check for missing data
imputer = SimpleImputer(strategy='median')
genotypes_imputed = pd.DataFrame(imputer.fit_transform(genotypes_pruned), columns=genotypes_pruned.columns)
phenotype_imputed = pd.DataFrame(imputer.fit_transform(phenotype), columns=phenotype.columns)

# Scale the data
scaler = StandardScaler()
genotypes_scaled = pd.DataFrame(scaler.fit_transform(genotypes_imputed), columns=genotypes_pruned.columns)

# Check for any remaining NaNs or infinities
genotypes_scaled = genotypes_scaled.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
phenotype_scaled = pd.DataFrame(scaler.fit_transform(phenotype_imputed), columns=phenotype.columns)

In [ ]:
# Extract the AVGROW97 column from the phenotype dataframe
y = phenotype_scaled['AVGROW97']
# Construct X from the genotype dataframe
X = genotypes_scaled

# Determine the number of samples (rows) and features (columns)
num_samples, num_features = X.shape

print(f"Number of samples: {num_samples}")
print(f"Number of features: {num_features}")

Number of samples: 291
Number of features: 243


### 6. GBM Feature Selection with Hyperparameter Tuning

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import KBinsDiscretizer

est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
y_binned = est.fit_transform(y.values.reshape(-1, 1)).ravel()

# GBM hyperparameters grid
gbm_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
}

# Randomized search with cross-validation
gbm_random_search = RandomizedSearchCV(GradientBoostingClassifier(random_state=0),
                                       gbm_params, n_iter=10, cv=5, random_state=0, error_score='raise')
gbm_random_search.fit(X, y_binned)
selected_features_gbm = np.argsort(gbm_random_search.best_estimator_.feature_importances_)[::-1]
# Save selected features from GBM
np.savetxt("selected_features_gbm.txt", selected_features_gbm, fmt='%d')
# GBM feature importances
np.savetxt("gbm_importances.txt", gbm_random_search.best_estimator_.feature_importances_, fmt='%f')

In [ ]:
from google.colab import files

# Download the files to your local system
files.download('selected_features_gbm.txt')
files.download('gbm_importances.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>